<a href="https://colab.research.google.com/github/justinfmccarty/epwmorph/blob/main/morphing_routines.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [182]:
!pip install meteocalc
!pip install skyfield

     |████████████████████████████████| 389kB 9.8MB/s 
     |████████████████████████████████| 40kB 5.6MB/s 
     |████████████████████████████████| 256kB 14.1MB/s 
  Created wheel for skyfield: filename=skyfield-1.34-cp36-none-any.whl size=421399 sha256=14fa132eec4382b8f5d660de0a3ffa52fe3fe2b44478539a68b4095aaac5c0fd
  Stored in directory: /root/.cache/pip/wheels/48/e1/ad/9e7673f3ea6061030e82535ce33a14d79fd3b54290835c60c8
  Created wheel for jplephem: filename=jplephem-2.15-cp36-none-any.whl size=46018 sha256=077a89035b5d7d3c169c0b0c5e93ef4991aea24cb88ab6b012b9f6dff72ba87f
  Stored in directory: /root/.cache/pip/wheels/01/71/9c/beaa33beb607ef67b62765324a10cd3584e30fbeef8856b7a8
Successfully built skyfield jplephem


In [259]:
import pandas as pd
# import metpy.calc as mpcalc
# from metpy.units import units
import numpy as np
from datetime import timedelta
import datetime as dt
import math
from meteocalc import dew_point as calcdewpt

from skyfield import api, almanac

In [3]:
def epw_to_dataframe(weather_path):
    epw_labels = ['year', 'month', 'day', 'hour', 'minute', 'datasource', 'drybulb_C', 'dewpoint_C', 'relhum_percent',
                  'atmos_Pa', 'exthorrad_Whm2', 'extdirrad_Whm2', 'horirsky_Whm2', 'glohorrad_Whm2',
                  'dirnorrad_Whm2', 'difhorrad_Whm2', 'glohorillum_lux', 'dirnorillum_lux', 'difhorillum_lux',
                  'zenlum_lux', 'winddir_deg', 'windspd_ms', 'totskycvr_tenths', 'opaqskycvr_tenths', 'visibility_km',
                  'ceiling_hgt_m', 'presweathobs', 'presweathcodes', 'precip_wtr_mm', 'aerosol_opt_thousandths',
                  'snowdepth_cm', 'days_last_snow', 'Albedo', 'liq_precip_depth_mm', 'liq_precip_rate_Hour']

    df = pd.DataFrame(pd.read_csv(weather_path, skiprows=8, header=None, names=epw_labels).drop('datasource', axis=1))
    return df



In [533]:
df = epw_to_dataframe('/content/CAN_BC_Vancouver.Intl.AP.718920_CWEC2016.epw')
# returns datetime objects
df.index = pd.to_datetime(df.apply(lambda row: dt.datetime(2011, 
                                                           int(row.month), 
                                                           int(row.day), 
                                                           int(row.hour)-1,
                                                           int(row.minute)), axis=1))
df.head()

,year,month,day,hour,minute,drybulb_C,dewpoint_C,relhum_percent,atmos_Pa,exthorrad_Whm2,extdirrad_Whm2,horirsky_Whm2,glohorrad_Whm2,dirnorrad_Whm2,difhorrad_Whm2,glohorillum_lux,dirnorillum_lux,difhorillum_lux,zenlum_lux,winddir_deg,windspd_ms,totskycvr_tenths,opaqskycvr_tenths,visibility_km,ceiling_hgt_m,presweathobs,presweathcodes,precip_wtr_mm,aerosol_opt_thousandths,snowdepth_cm,days_last_snow,Albedo,liq_precip_depth_mm,liq_precip_rate_Hour
2011-01-01 00:00:00,2011,1,1,1,0,-1.8,-4.5,80,102160,0,0,238,0,0,0,0,0,0,0,0,0.0,0,0,48.3,77770,0,999999990,0,0.085,0,88,0.2,0.0,0.0
2011-01-01 01:00:00,2011,1,1,2,0,-3.0,-5.3,82,102010,0,0,233,0,0,0,0,0,0,0,0,0.0,0,0,48.3,77770,0,999999990,0,0.085,0,88,0.2,0.0,0.0
2011-01-01 02:00:00,2011,1,1,3,0,-4.0,-6.0,84,101920,0,0,229,0,0,0,0,0,0,0,0,0.0,0,0,48.3,77770,0,999999990,0,0.085,0,88,0.2,0.0,0.0
2011-01-01 03:00:00,2011,1,1,4,0,-4.7,-6.4,86,101880,0,0,226,0,0,0,0,0,0,0,0,0.0,0,0,48.3,77770,0,999999990,0,0.085,0,88,0.2,0.0,0.0
2011-01-01 04:00:00,2011,1,1,5,0,-5.2,-6.7,88,101880,0,0,224,0,0,0,0,0,0,0,0,0.0,0,0,48.3,77770,0,999999990,0,0.085,0,88,0.2,0.0,0.0


## Utilities

In [149]:
# the following are recreation of PVLIB 
# https://pvlib-python.readthedocs.io/en/stable/_modules/pvlib/solarposition.html
# or from ASU Solar Power Labs https://www.pveducation.org/pvcdrom/properties-of-sunlight/solar-time 
def simple_day_angle(dayofyear,offset=1):
  return ((2*np.pi)/365)*(dayofyear-offset)

def bday(dayangle):
  return dayangle - ((2*np.pi)/365) * 80

def equation_of_time(bday):
  return 9.87*np.sin(2*dbay)-7.53*np.cos(bday)-1.5*np.sin(bday)

def local_time_meridian(utc_offset):
  # requires the utc offset for local time zone
  # https://en.wikipedia.org/wiki/List_of_UTC_time_offsets
  return 15*utc_offset

def time_correction(longitude, local_time_meridian, equation_of_time):
  return 4 * (longitude - local_time_meridian) + equation_of_time

def local_solar_time(local_time, time_correction):
  return local_time + (time_correction / 60)


## Dry Bulb Temperature (dbt)

In [150]:


fut_tas = pd.Series([-5,3,5,9,13,15,17,19,24,15,11,-1])
fut_tmax = pd.Series([3,13,15,19,22,25,27,29,33,25,19,10])
fut_tmin = pd.Series([-6,1,3,6,9,10,11,10,18,13,8,-4])
hist_tas = pd.Series([-7,0,2,6,10,12,15,18,21,14,6,-3])
hist_tmax = pd.Series([2,11,10,14,20,22,25,27,26,18,13,5])
hist_tmin = pd.Series([-12,-5,-3,0,1,5,6,10,12,11,0,-10])


def means(series):
  max = series.resample('D').max().resample('M').mean().reset_index(drop=True)
  min = series.resample('D').min().resample('M').mean().reset_index(drop=True)
  mean = series.resample('D').mean().resample('M').mean().reset_index(drop=True)
  return max, min, mean

def change(fut_tas,hist_tas,fut_tmax,hist_tmax,fut_tmin,hist_tmin):
  tas = fut_tas-hist_tas
  tmax = fut_tmax-hist_tmax
  tmin = fut_tmin-hist_tmin
  return tas, tmax, tmin

def morph_dbt(df, fut_tas,hist_tas,fut_tmax,hist_tmax,fut_tmin,hist_tmin):
  months = list(range(1, 12 + 1, 1))
  dbt_max_mean, dbt_min_mean, dbt_mean = means(df['drybulb_C'])
  tas_change, tmax_change, tmin_change = change(fut_tas, hist_tas,
                                                fut_tmax, hist_tmax,
                                                fut_tmin, hist_tmin)
  tas_change = dict(zip(months, tas_change)) 
  dbt_scale = dict(zip(months, (tmax_change - tmin_change) / (dbt_max_mean - dbt_min_mean)))
  dbt_mean = dict(zip(months, dbt_mean))
  return df.apply(lambda x: x['drybulb_C'] + tas_change[x['month']] + dbt_scale[x['month']] * (x['drybulb_C'] - dbt_mean[x['month']]),axis=1).rename("drybulb_C")

new_dbt = morph_dbt(df, fut_tas,hist_tas,fut_tmax,hist_tmax,fut_tmin,hist_tmin)


## Relative Humidity (relhum)

In [151]:
fut_rh = pd.Series([35,45,55,55,78,77,95,65,70,60,80,70])
hist_rh = pd.Series([40,60,54,50,76,70,88,61,72,64,85,75])

def morph_relhum(df, fut_relhum, hist_relhum):
    # requires fut_ and hist_ inputs to be monthly climatologies
    months = list(range(1,12+1,1))
    relhum_change = dict(zip(months, fut_relhum - hist_relhum))
    return df.apply(lambda x: x['relhum_percent'] + relhum_change[x['month']],axis=1).rename("relhum_percent")

new_rh = np.clip(morph_relhum(df, fut_rh, hist_rh),0,100).astype(int)

## Pressure (pr)

In [152]:
fut_pr = pd.Series([101860,101865,101880,101880,102160,102230,101890,101950,102110,101890,101980,102380])
hist_pr = pd.Series([101890,101855,101870,101830,102100,101880,101880,101890,102160,101880,101865,102180])

def morph_pr(df, fut_pr, hist_pr):
    # requires fut_ and hist_ inputs to be monthly climatologies
    months = list(range(1,12+1,1))
    pr_change = dict(zip(months, fut_pr - hist_pr))
    return df.apply(lambda x: x['atmos_Pa'] + pr_change[x['month']],axis=1).rename("atmos_Pa")

new_pr = morph_pr(df, fut_pr, hist_pr)

## Dew Point (dewpt)

In [153]:
# def calc_sat_pr(pressure, dbt, rel_hum):
#     # pressure units in in Pa
#     # pressure units out in Pa
#     # temp units in C
#     mixing = mpcalc.mixing_ratio_from_relative_humidity(pressure*units.Pa,
#                                                         dbt*units.degC,
#                                                         rel_hum*units.percent)
#     return mpcalc.vapor_pressure(pressure*units.Pa, mixing)

# def calc_partial_water_pr(pressure, dbt, rel_hum):
#     # pressure units in in Pa
#     # pressure units out in kPa
#     # temp units in C
#     # rel hum in %
#     return (rel_hum * calc_sat_pr(pressure, dbt, rel_hum)) / 1000

# def morph_dewpt(epw_dewpt, epw_pressure, epw_dbt, epw_rh):
#     pw = calc_partial_water_pr(epw_pressure, epw_dbt, epw_rh)
#     if epw_dewpt>=0:
#         epw_dewpt_fut = 6.54 + 14.526 + np.log(pw) + 0.7389 * np.log(pw)**2 + 0.09486 * np.log(pw)**3 + 0.4569 * pw**0.1984
#     else:
#         epw_dewpt_fut = 6.09 + 12.608 * np.log(pw) + 0.4959 * np.log(pw)**2
#     return epw_dewpt_fut

# TODO THe above can be used to more accuratley account for colder temperatures in dew point calc

def morph_dewpt(fut_dbt,fut_rh):
  df = pd.concat([fut_dbt,fut_rh],axis=1)
  return round(df.apply(lambda x: calcdewpt(x[0], x[1]),axis=1).astype(float),1)

new_dewpt = morph_dewpt(new_dbt,new_rh)

### Working On Dewpoint V2

In [541]:
df_new = pd.concat([new_dbt,new_rh,new_pr,new_dewpt],axis=1)

In [ ]:
df_old = df[['drybulb_C','atmos_Pa','relhum_percent','dewpoint_C']].copy()
# df_old['dewpt2'] = df_old.apply(lambda x: calcdewpt(x['drybulb_C'], x['relhum_percent']),axis=1)
old_dbt = df_old['drybulb_C']
old_rh = df_old['relhum_percent']
df_old['dpt2'] = round(morph_dewpt(old_dbt,old_rh).astype(float),1)


In [107]:
def calcdew_above(pw):
  return 6.54 + 14.526 + np.log(pw) + 0.7389 * np.log(pw)**2 + 0.09486 * np.log(pw)**3 + 0.4569 * pw**0.1984

def calcdew_sub(pw):
  return 6.09 + 12.608 * np.log(pw) + 0.4959 * np.log(pw)**2

In [101]:
# calc_partial_water_pr(df_new['atmos_Pa'][2], df_new['drybulb_C'][2], df_new['relhum_percent'][2])
df_new['dewpt'] = df_new.apply(lambda x: calcdewpt(x['drybulb_C'], x['relhum_percent']),axis=1)
df_new['vapor'] = df_new.apply(lambda x: calc_sat_pr(x['atmos_Pa'], x['drybulb_C'], x['relhum_percent']),axis=1)
df_new['partial'] = df_new.apply(lambda x: calc_partial_water_pr(x['atmos_Pa'], x['drybulb_C'], x['relhum_percent']),axis=1)

In [ ]:
df_new['old_dp'] = df['dewpoint_C']


In [ ]:
df_new['dewpoint_met'] = np.where(df_new['old_dp']>=0,
                                  df_new['partial'].apply(lambda x: calcdew_above(x.magnitude)),
                                  df_new['partial'].apply(lambda x: calcdew_sub(x.magnitude)))
df_new.iloc[3000:3024]

In [ ]:
df_old = df[['drybulb_C','atmos_Pa','relhum_percent','dewpoint_C']].copy()
df_old['dewpt2'] = df_old.apply(lambda x: calcdewpt(x['drybulb_C'], x['relhum_percent']),axis=1)
# df_old['vapor'] = df_old.apply(lambda x: calc_sat_pr(x['atmos_Pa'], x['drybulb_C'], x['relhum_percent']),axis=1)
# df_old['partial'] = (df_old['vapor']*(df_old['relhum_percent']/100))

# df_old['dewpoint_met'] = np.where(df_old['dewpoint_C']>=0,
#                                   df_old['partial'].apply(lambda x: calcdew_above(x.magnitude)),
#                                   df_old['partial'].apply(lambda x: calcdew_sub(x.magnitude)))


## Global Horizontal Radiation (glhor)

In [154]:
fut_glhor = pd.Series([33,64,111,190,220,280,230,210,140,85,38,24]) #watt per m2
hist_glhor = pd.Series([36,67,109,184,208,244,243,218,146,87,39,26]) #watt per m2

def morph_glohr(df, hist_glhor, fut_glhor):
  df['single'] = 1
  months = list(range(1,12+1,1))

  month_hours = dict(zip(months, df['single'].resample('M').sum().tolist())) #hours
  month_glhor = dict(zip(months, df['glohorrad_Whm2'].resample('M').sum().tolist())) #watt-hours per m2 
  month_glhor_mean_list = []
  for key in month_hours:
    mean = (month_glhor[key])/month_hours[key]
    month_glhor_mean_list.append(mean)
  month_glhor_mean_list = dict(zip(months, month_glhor_mean_list)) #watt per m2 

  glhor_change = dict(zip(months, fut_glhor - hist_glhor))
  glhor_scale_list = []
  for key in month_glhor_mean_list:
    glhor_scale = 1 + (glhor_change[key]/month_glhor_mean_list[key])
    glhor_scale_list.append(glhor_scale)
  glhor_scale_list = dict(zip(months, glhor_scale_list))

  return df.apply(lambda x: x['glohorrad_Whm2'] * glhor_scale_list[x['month']],axis=1).rename("glohorrad_Whm2")

new_glohr = morph_glohr(df, hist_glhor, fut_glhor)

## Diffuse Horizontal Radiation (dfhor)

In [177]:
hours = list(range(1,8760+1,1))
days = list(range(1,365+1,1))
months = list(range(1,12+1,1))

clearness = pd.Series(new_glohr / df['exthorrad_Whm2']).rename("clearness").fillna(0)
clearness_daily = clearness.resample('D').sum()

clearness_list = dict(zip(hours, clearness.tolist()))
clearness_day_list = dict(zip(days,clearness_daily.tolist()))

In [566]:
#define sunrise sunset
latitude = 49.264703
longitude = -123.148943

ts = api.load.timescale()
eph = api.load('de421.bsp')

location = api.Topos(latitude, longitude)

t0 = ts.utc(2011-1, 12, 31, 0)
t1 = ts.utc(2011+1, 1, 2, 0)

t, y = almanac.find_discrete(t0, t1, almanac.sunrise_sunset(eph, location))
og = df.copy()
times = pd.Series(t.utc_datetime()).rename('datetimes')
times = times + timedelta(hours=-8, minutes=0)
keys = pd.Series(y).rename('Rise_Set')
keys = pd.Series(np.where(keys==0,'Sunset','Sunrise')).rename('Rise_Set')
join = pd.concat([times,keys],axis=1)
join.set_index(join['datetimes'], inplace=True)
join['year'] = join['datetimes'].dt.year
join['month'] = join['datetimes'].dt.month
join['day'] = join['datetimes'].dt.day
join['hour'] = join['datetimes'].dt.hour
join['minute'] = 0
join = join[join['year']==2011]
join['Timestamp'] = join.apply(lambda row: dt.datetime(row.year, row.month, row.day, row.hour),axis=1)
join.set_index('Timestamp', inplace=True)

join_sub = pd.DataFrame(join['Rise_Set'])
join_sub['dtime']=join.index
og['dtime']=og.index
og = og.merge(join_sub,how='left', left_on='dtime', right_on='dtime')
og['Rise_Set'] = og['Rise_Set'].fillna('Neither')
og.set_index(og['dtime'], inplace=True)
rise_set = og['Rise_Set']

In [567]:
rise_set

dtime
2011-01-01 00:00:00    Neither
2011-01-01 01:00:00    Neither
2011-01-01 02:00:00    Neither
2011-01-01 03:00:00    Neither
2011-01-01 04:00:00    Neither
                        ...   
2011-12-31 19:00:00    Neither
2011-12-31 20:00:00    Neither
2011-12-31 21:00:00    Neither
2011-12-31 22:00:00    Neither
2011-12-31 23:00:00    Neither
Name: Rise_Set, Length: 8760, dtype: object

## Failed Sunrise
the following was a failed and innefficient way to define sunrise/sunset

In [204]:

ts = api.load.timescale()
eph = api.load('de421.bsp')

[#################################] 100% de421.bsp


In [203]:
latitude = 49.264703
longitude = -123.148943

def change_coords(lat,lon):
  if lat >= 0:
    lat_dir = 'N'
  else:
    lat_dir = 'S'
  if lon >= 0:
    lon_dir = 'E'
  else:
    lon_dir = 'W'

location = api.Topos(latitude, longitude)
location

<Topos Earth latitude 49deg 15' 52.9" N longitude -123deg 08' 56.2" E>

In [334]:
t0 = ts.utc(2021, 1, 1, 0)
t1 = ts.utc(2021, 1, 1, 23)

f = almanac.dark_twilight_day(eph, location)
t, y = almanac.find_discrete(t0, t1, f)

# print(t.utc_iso()[4], y[4]) #start of day

# print(t.utc_iso()[0], y[0]) #start of night

# t.utc_datetime()[4] + timedelta(hours=-8, minutes=0)
# t.utc_datetime()[0] + timedelta(hours=-8, minutes=0)
for ti, yi in zip(t, y):
    print(yi, '<>', ti.utc_iso(), '<>', almanac.TWILIGHTS[yi])

3 <> 2021-01-01T00:24:15Z <> Civil twilight
2 <> 2021-01-01T01:01:41Z <> Nautical twilight
1 <> 2021-01-01T01:42:20Z <> Astronomical twilight
0 <> 2021-01-01T02:21:03Z <> Night
1 <> 2021-01-01T14:11:09Z <> Astronomical twilight
2 <> 2021-01-01T14:49:50Z <> Nautical twilight
3 <> 2021-01-01T15:30:26Z <> Civil twilight
4 <> 2021-01-01T16:07:48Z <> Day


In [ ]:
for i in list(range(1,23+1,1)):
  print(i)
  t0 = ts.utc(2021, 1, 1, i)
  t1 = ts.utc(2021, 1, 1, i+1)

  f = almanac.dark_twilight_day(eph, location)
  t, y = almanac.find_discrete(t0, t1, f)
  
  if y.size == 0:
    print('No')
    pass
  else:
    if almanac.TWILIGHTS[y[0]]=='Day':
      print('Yes')
      rise = t.utc_datetime()[0] + timedelta(hours=-8, minutes=0)
      print('Sunrise is at {HOUR}:{MIN}'.format(HOUR=rise.hour,MIN=rise.minute))
    else:
      print('No')



In [ ]:
for i in list(range(1,23+1,1)):
  print(i)
  t0 = ts.utc(2021, 1, 1, i)
  t1 = ts.utc(2021, 1, 1, i+1)

  f = almanac.dark_twilight_day(eph, location)
  t, y = almanac.find_discrete(t0, t1, f)
  
  if y.size == 0:
    print('No')
    pass
  else:
    if almanac.TWILIGHTS[y[0]]=='Night':
      print('Yes')
      sunset = t.utc_datetime()[0] + timedelta(hours=-8, minutes=0)
      print('Sunset is at {HOUR}:{MIN}'.format(HOUR=sunset.hour,MIN=sunset.minute))
    else:
      print('No')

In [ ]:
# this was super innefficient 

def (year, month, day, hour):
  t0 = ts.utc(year, month, day, hour)
  t1 = ts.utc(year, month, day, hour+1)

  f = almanac.dark_twilight_day(eph, location)
  t, y = almanac.find_discrete(t0, t1, f)
  
  if y.size == 0:
    return 'FALSE'
  elif almanac.TWILIGHTS[y[0]]=='Day':
    return 'TRUE'
  else:
    return 'FALSE'

idx = df.index + timedelta(hours=-8, minutes=0)
solar_df = pd.DataFrame()
solar_df['datetimes'] = idx
solar_df = solar_df.set_index(solar_df['datetimes'])
solar_df['year'] = solar_df['datetimes'].dt.year
solar_df['month'] = solar_df['datetimes'].dt.month
solar_df['day'] = solar_df['datetimes'].dt.day
solar_df['hour'] = solar_df['datetimes'].dt.hour
solar_df['minute'] = solar_df['datetimes'].dt.minute
solar_df['sunrise'] = solar_df.apply(lambda x: check_sunrise(x['year'],x['month'],x['day'],x['hour']),axis=1)

In [272]:
t.utc_datetime()[0].hour + 1

def check_sunset(year,month,day,hour):
  t0 = ts.utc(year, month, day, hour)
  t1 = ts.utc(year, month, day, hour+1)

  f = almanac.dark_twilight_day(eph, location)
  t, y = almanac.find_discrete(t0, t1, f)

1

## Direct Normal Radiation (dnor)